In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso,Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [6]:
df = pd.read_csv('data/stud.csv')

In [7]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [8]:
X = df.drop(columns=['math_score'], axis = 1)

In [9]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [10]:
Y = df['math_score']

In [11]:
Y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [12]:
print("Categories in 'gender' variable:     ",end=" " )
print(df['gender'].unique())

print("Categories in 'race_ethnicity' variable:  ",end=" ")
print(df['race_ethnicity'].unique())

print("Categories in'parental level of education' variable:",end=" " )
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable:     ",end=" " )
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     ",end=" " )
print(df['test_preparation_course'].unique())


Categories in 'gender' variable:      ['female' 'male']
Categories in 'race_ethnicity' variable:   ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in'parental level of education' variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable:      ['standard' 'free/reduced']
Categories in 'test preparation course' variable:      ['none' 'completed']


In [19]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)


In [20]:
X = preprocessor.fit_transform(X)

In [22]:
X.shape

(1000, 19)

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=40)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [27]:
def evaluate_model(true, predicted):
    mse = mean_squared_error(true, predicted)
    mae = mean_absolute_error(true, predicted)
    r2_square = r2_score(true, predicted)
    rmse = np.sqrt(mse)
    return r2_square, mae, rmse

Compare multiple linear models to choose one from them based on r2_score

In [33]:
models = {
    "LinearRegression" : LinearRegression(),
    "LassoRegression" : Lasso(),
    "RidgeRegression" : Ridge()
}

model_lst = []
r2_lst = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, Y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(Y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(Y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_lst.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_lst.append(model_test_r2)
    

LinearRegression
Model performance for Training set
- Root Mean Squared Error: 4.1838
- Mean Absolute Error: 0.8810
- R2 Score: 5.2568
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4.5371
- Mean Absolute Error: 0.8538
- R2 Score: 5.6593
LassoRegression
Model performance for Training set
- Root Mean Squared Error: 5.1214
- Mean Absolute Error: 0.8170
- R2 Score: 6.5207
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.4091
- Mean Absolute Error: 0.7940
- R2 Score: 6.7188
RidgeRegression
Model performance for Training set
- Root Mean Squared Error: 4.1823
- Mean Absolute Error: 0.8810
- R2 Score: 5.2570
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4.5345
- Mean Absolute Error: 0.8540
- R2 Score: 5.6569


In [35]:
pd.DataFrame(list(zip(model_lst, r2_lst)), columns=["MOdel", "R2 Score"]).sort_values("R2 Score", ascending=False)


,MOdel,R2 Score
1,LassoRegression,6.718754
0,LinearRegression,5.659252
2,RidgeRegression,5.656945


In [38]:
Lasso_model = Lasso(alpha=0.01)
Lasso_model.fit(X_train, Y_train)
y_pred = Lasso_model.predict(X_test)
r2, mae, rmse = evaluate_model(Y_test, y_pred)
score = r2_score(Y_test, y_pred)
print("Accuracy of the model:", score)

Accuracy of the model: 0.8545586362037714
